<a href="https://colab.research.google.com/github/louiewang820/car-condition-classification/blob/main/car_condition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
!git status


fatal: not a git repository (or any of the parent directories): .git
